In [ ]:
import numpy as np
import sklearn

In [ ]:
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

# 3D 데이터 생성
angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [ ]:
# SVD를 이용해 주성분 구하기
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

In [ ]:
# 차원 투영
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

In [ ]:
# 복원
X_recov_centered = X2D.dot(Vt.T[:,:2].T)
X_recov = X_recov_centered + X.mean(axis=0)
print(X_recov[:3])

[[-1.01450604 -0.54656333 -0.27441525]
 [-0.02103231  0.55108376  0.18101894]
 [-0.95379477 -0.4668077  -0.24237013]]


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)   # PCA모델을 사용해 데이터셋 차원을 2로 줄임
X2D = pca.fit_transform(X)

In [ ]:
pca.explained_variance_ratio_  # 변수에 저장된 주성분의 설명된 분산의 비율
# 데이터셋 분산의 84.2%가 첫번째 PC를 따라 놓여있고 14.6%가 두번째 PC를 따라 놓여있다.

array([0.84248607, 0.14631839])

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()   # 데이터를 2차원으로 줄임
pca.fit(X)
vr = pca.explained_variance_ratio_
cumsum = np.cumsum(vr)
d = np.argmax(cumsum >= 0.95) +1

print(vr)   # 모든 분산 값
print(np.sum(vr))   # 데이터셋 분산을 모두 더하면 100
print(cumsum)   # 분산을 순차적으로 더한 값
print(d)    # 차원의 수

[0.84248607 0.14631839 0.01119554]
0.9999999999999999
[0.84248607 0.98880446 1.        ]
2


In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)   # 사이킷런에서 mnist 데이터 불러오기
mnist.target = mnist.target.astype(np.uint8)

In [ ]:
from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)   #  사이킷런으로 데이터 분리

In [ ]:
pca = PCA(n_components=0.95)    # n_components=0.95 :  95퍼의 분산의 비율을 보존
X_reduced = pca.fit_transform(X_train)

In [ ]:
pca = PCA(n_components=154)     # mnist 의 데이터셋을 154차원으로 압축
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)  # 784차원으로 복구

In [ ]:
rnd_pca = PCA(n_components=154, svd_solver="randomized", random_state=42)   # 확률적 알고리즘응ㄹ 사용해 처음 d개의 주성분에 대한 근삿값 찾기
X_reduced = rnd_pca.fit_transform(X_train)

In [ ]:
from sklearn.decomposition import IncrementalPCA    # 점진적 PCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)      # 154차원으로 차원 축소
for X_batch in np.array_split(X_train, n_batches):      # 100개의 미니배치로 나눔
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

In [ ]:
filename = "my_mnist.data"      # mnist데이터 복사
m, n = X_train.shape

X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))     # 넘파이의 memmap 파이썬 클래스를 사용해 하드 디스크의 이진 파일에 저장된 매우 큰 배열을 메모리에 들어 있는것처럼 다룸

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

IncrementalPCA(batch_size=525, copy=True, n_components=154, whiten=False)

In [ ]:
from sklearn.datasets import make_swiss_roll    # 사이킷런에서 스위스롤 데이터 불러오기

X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
y = t > 6.9

In [ ]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)   # KernelPCA를 사용해 RBF 커널로 kPCA 적용
X_reduced = rbf_pca.fit_transform(X)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),    # 2차원으로 축소
        ("log_reg", LogisticRegression(solver="lbfgs"))     #분류를 위해 로지스틱 회귀 적용
    ])

param_grid = [{
        "kpca__gamma": np.linspace(0.03, 0.05, 10),
        "kpca__kernel": ["rbf", "sigmoid"]
    }]

grid_search = GridSearchCV(clf, param_grid, cv=3)   # GridSearchCV를 사용해 kPCA의 가장 좋응ㄴ 커널과 gamma 파라미터 탐색
grid_search.fit(X, y)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('kpca',
                                        KernelPCA(alpha=1.0, coef0=1,
                                                  copy_X=True, degree=3,
                                                  eigen_solver='auto',
                                                  fit_inverse_transform=False,
                                                  gamma=None, kernel='linear',
                                                  kernel_params=None,
                                                  max_iter=None, n_components=2,
                                                  n_jobs=None,
                                                  random_state=None,
                                                  remove_zero_eig=False,
                                                  tol=0)),
                                       ('log_reg',
                                 

In [ ]:
print(grid_search.best_params_)     # 가장 좋은 커널과 하이퍼파라미터는 best_params_ 변수에 저장됨

{'kpca__gamma': 0.043333333333333335, 'kpca__kernel': 'rbf'}


In [ ]:
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433,     # 가장 좋은 커널과 하이퍼파라미터 적용
                    fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)       # fit_inverse_transform=True : 재구성 원상 구하기

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(X, X_preimage)   # 재구성 원상 오차 계산

32.786308795766125

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding     # 지역 선형 임베딩

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)   # 각 훈련 샘플이 가장 가까운 이웃에 얼마나 선형적으로 연관되어 있는지 측정
X_reduced = lle.fit_transform(X)